In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/transformer')

In [5]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 35.8 MB/s eta 0:00:00


In [9]:
import torch
import pandas as pd
from models import Transformer
from transformers import AutoTokenizer
from train import evaluate
import torch.nn as nn
import numpy as np
from collections import Counter
from nltk import ngrams
from tqdm.notebook import tqdm
import math
# from torchtext.data.metrics import bleu_score

In [10]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
tokenizer.add_special_tokens({'bos_token': '<s>'})

c:\Users\leeht\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

In [11]:
vocab_size = tokenizer.vocab_size + 1
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 120
dropout = 0.1
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model_10 = Transformer(tokenizer, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, batch_size, device).to(device)
model_10.load_state_dict(torch.load('transformers_english_to_french_10.pt', map_location=device))

C:\Users\user\AppData\Local\Temp\ipykernel_26588\2375610529.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_10.load_state_dict(torch.load('transformers_english_to_

<All keys matched successfully>

In [ ]:
model_20 = Transformer(tokenizer, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, batch_size, device).to(device)
model_20.load_state_dict(torch.load('transformers_english_to_french_20.pt', map_location=device))

C:\Users\user\AppData\Local\Temp\ipykernel_26588\4023960284.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_20.load_state_dict(torch.load('transformers_english_to_

<All keys matched successfully>

In [5]:
model_30 = Transformer(tokenizer, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, batch_size, device).to(device)
model_30.load_state_dict(torch.load('transformers_english_to_french_30.pt', map_location=device))

C:\Users\leeht\AppData\Local\Temp\ipykernel_17780\3569055596.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_30.load_state_dict(torch.load('transformers_english_to

<All keys matched successfully>

In [6]:
model_30

Transformer(
  (encoder_embedding): InputEmbeddings(
    (embedding): Embedding(59515, 512)
  )
  (decoder_embedding): InputEmbeddings(
    (embedding): Embedding(59515, 512)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttentionLayer(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): PositionwiseFeedForward(
        (fc_1): Linear(in_features=512, out_features=2048, bias=True)
        (fc_2): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((512,), eps=1e-05, eleme

In [14]:
# translate 함수 정의
def translate(model, tokenizer, src_sentence, max_len, device='cuda'):
    model.eval()

    src_tokens = tokenizer.encode(src_sentence, return_tensors='pt').to(device)
    trg_tokens = torch.tensor([[tokenizer.bos_token_id]], dtype=torch.long, device=device)

    for _ in range(max_len):
        
        with torch.no_grad():
            output, attention = model(src_tokens, trg_tokens)

        next_token_logits = output[:, -1, :]
        next_token_id = next_token_logits.argmax(1).item()

        trg_tokens = torch.cat([trg_tokens, torch.tensor([[next_token_id]], dtype=torch.long, device=device)], dim=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    translated_sentence = tokenizer.decode(trg_tokens.squeeze().tolist(), skip_special_tokens=True)

    return translated_sentence, attention

In [8]:
test_df = pd.read_csv('test_preprocess.csv')
test_df

,src,tar
0,They say that Firefox is downloaded over 8 mil...,"Ils disent que chaque jour, Firefox est téléch..."
1,I really like being with you.,J'apprécie vraiment d'être avec toi.
2,"When I'm hot, a glass of cool water really ref...","Lorsque j'ai chaud, un verre d'eau fraîche me ..."
3,That's what I'm supposed to be doing.,C'est ce que je suis censée être en train de f...
4,"She was superstitious, as the people of that p...","Elle était superstitieuse, à l'instar des gens..."
...,...,...
23269,You dropped something.,Vous avez laissé tomber quelque chose.
23270,I need you right now.,J'ai besoin de vous sur-le-champ.
23271,The weather service has issued a severe weathe...,Le service météorologique a émis un avis de te...
23272,Let's do the math.,Faisons le calcul.


In [ ]:
test_df['tar'][0]

'Ils disent que chaque jour, Firefox est téléchargé plus de 8 millions de fois.'

In [ ]:
src_sentence = "They say that Firefox is downloaded over 8 million times a day."
translated_sentence, attention = translate(model_10, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: They say that Firefox is downloaded over 8 million times a day.
Translated Sentence: Ils peut passer quelques années jouères de la plus grande ans.


In [ ]:
src_sentence = "They say that Firefox is downloaded over 8 million times a day."
translated_sentence, attention = translate(model_20, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: They say that Firefox is downloaded over 8 million times a day.
Translated Sentence: Ils ditent que les vêtements sont remis à un grand-cent-cent-cent-cinq ans.


In [ ]:
src_sentence = "They say that Firefox is downloaded over 8 million times a day."
translated_sentence, attention = translate(model_30, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: They say that Firefox is downloaded over 8 million times a day.
Translated Sentence: On dit que les qualités de téléphone sont très légumes pendant longtemps par jour.


In [ ]:
test_df['src'][1]

'I really like being with you.'

In [ ]:
test_df['tar'][1]

"J'apprécie vraiment d'être avec toi."

In [ ]:
src_sentence = "I really like being with you."
translated_sentence, attention = translate(model_10, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: I really like being with you.
Translated Sentence: J'aime vraiment être avec vous.


In [ ]:
src_sentence = "I really like being with you."
translated_sentence, attention = translate(model_20, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: I really like being with you.
Translated Sentence: J'aime vraiment être avec vous.


In [ ]:
src_sentence = "I really like being with you."
translated_sentence, attention = translate(model_30, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: I really like being with you.
Translated Sentence: J'aime vraiment être avec vous.


In [ ]:
test_df['src'][2]

"When I'm hot, a glass of cool water really refreshes me."

In [ ]:
test_df['tar'][2]

"Lorsque j'ai chaud, un verre d'eau fraîche me rafraîchit vraiment."

In [ ]:
src_sentence = "When I'm hot, a glass of cool water really refreshes me."
translated_sentence, attention = translate(model_10, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: When I'm hot, a glass of cool water really refreshes me.
Translated Sentence: Lorsque je me suis très appelé, je me suis très intéressée.


In [ ]:
src_sentence = "When I'm hot, a glass of cool water really refreshes me."
translated_sentence, attention = translate(model_20, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: When I'm hot, a glass of cool water really refreshes me.
Translated Sentence: Lorsque je porte chaud, il faut de têtes bien m'asseoir.


In [ ]:
src_sentence = "When I'm hot, a glass of cool water really refreshes me."
translated_sentence, attention = translate(model_30, tokenizer, src_sentence, max_seq_length, device=device)
print("Source Sentence:", src_sentence)
print("Translated Sentence:", translated_sentence)

Source Sentence: When I'm hot, a glass of cool water really refreshes me.
Translated Sentence: Lorsque je suis chaud, un verre d'eau fraîche-moi vraiment.


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.src = self.tokenizer(list(self.data['src']), padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        self.tar = self.tokenizer(['<s>' + s for s in self.data['tar']], padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.src[idx], self.tar[idx]

In [ ]:
test_ds = CustomDataset(test_df, tokenizer, 120)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64)
test_dl

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
test_loss = evaluate(model_10, test_dl, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

valid batch iteration:   0%|          | 0/364 [00:00<?, ?it/s]

Test Loss: 1.568 | Test PPL: 4.797


In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
test_loss = evaluate(model_20, test_dl, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

valid batch iteration:   0%|          | 0/364 [00:00<?, ?it/s]

Test Loss: 0.962 | Test PPL: 2.618


In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
test_loss = evaluate(model_30, test_dl, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

valid batch iteration:   0%|          | 0/364 [00:00<?, ?it/s]

Test Loss: 0.748 | Test PPL: 2.113


In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data.iloc[idx, 0], self.data.iloc[idx, 1]

test_ds = CustomDataset(test_df)

In [12]:
def calc_bleu_score(DS, model, tokenizer, max_seq_length, device):
    trgs = []
    preds = []
    translated_texts = []
    for i, (src_text, trg_text) in tqdm(enumerate(DS), total = len(DS)):

        translated_text, _ = translate(model, tokenizer, src_text, max_seq_length, device=device)
        translated_texts.append(translated_text)
        trg = tokenizer.tokenize(trg_text)
        translated_tok = tokenizer.tokenize(translated_text)

        trgs += [[trg]]
        preds += [translated_tok]

        if (i + 1) % 1000 == 0:
            print(f"[{i + 1}/{len(DS)}]")
            print(f"입력: {src_text}")
            print(f"정답: {trg_text}")
            print(f"AI의 번역: {translated_text}")
    # 여러 문장을 묶어서 BLEU score에 넘겨줌
    bleu1 = bleu_score(preds, trgs, max_n = 4, weights = [1,0,0,0])
    bleu2 = bleu_score(preds, trgs, max_n = 4, weights = [0,1,0,0])
    bleu3 = bleu_score(preds, trgs, max_n = 4, weights = [0,0,1,0])
    bleu4 = bleu_score(preds, trgs, max_n = 4, weights = [0,0,0,1])
    bleu = bleu_score(preds, trgs)
    print()
    print(f'Total BLEU Score = {bleu*100:.2f}')
    print(f'1-gram BLEU Score = {bleu1*100:.2f}')
    print(f'2-gram BLEU Score = {bleu2*100:.2f}')
    print(f'3-gram BLEU Score = {bleu3*100:.2f}')
    print(f'4-gram BLEU Score = {bleu4*100:.2f}')
    return translated_texts, bleu1, bleu2, bleu3, bleu4, bleu

In [13]:
translated_texts, bleu1, bleu2, bleu3, bleu4, bleu = calc_bleu_score(test_ds, model_30, tokenizer, max_seq_length, device)

  0%|          | 0/23274 [00:00<?, ?it/s]

[1000/23274]
입력: Everybody makes mistakes, don't they?
정답: Tout le monde commet des erreurs, pas vrai ?
AI의 번역: Tout le monde fait des erreurs, n'est-ce pas?
[2000/23274]
입력: We used to be best friends.
정답: Nous étions les meilleurs amis.
AI의 번역: Nous étions meilleures amis.
[3000/23274]
입력: We have to get to the hospital.
정답: Nous devons nous rendre à l'hôpital.
AI의 번역: Nous devons aller à l'hôpital.
[4000/23274]
입력: Tom didn't even know who that was.
정답: Tom ne savait même pas qui c'était.
AI의 번역: Tom ne savait même pas qui était cela.
[5000/23274]
입력: Any play opening to bad reviews won't last long.
정답: Toute pièce qui ouvre sur de mauvaises critiques ne fait pas long feu.
AI의 번역: N'importe quelle partie de la mauvaise humeur ne s'est pas échappé depuis longtemps.
[6000/23274]
입력: Do you play in a band?
정답: Joues-tu dans un groupe ?
AI의 번역: Jouez-vous dans un groupe?
[7000/23274]
입력: This place has a mysterious atmosphere to it.
정답: L'atmosphère de cet endroit est mystérieuse.
AI의 번

In [14]:
translated_texts

['On dit que les qualités de téléphone sont très légumes pendant longtemps par jour.',
 "J'aime vraiment être avec vous.",
 "Lorsque je suis chaud, un verre d'eau fraîche-moi vraiment.",
 "C'est ce que je suis censé faire.",
 "C'était une grosse personne extraordinaire, plusieurs gens étaient habituellement.",
 "C'était notre nôtre.",
 "Je dois me rappeler d'écrire ça.",
 "Ce n'est pas facile de comprendre ses idées.",
 'Il me rend fou.',
 'Vous ai-je fait attendre?',
 'Ça me va bien.',
 'Plusieurs amis ont été élèves au Japon.',
 'Tom nous a aidé à pousser la voiture.',
 'Ce médicament aura lieu de ton mal.',
 "C'est un fait.",
 'Je suis sûr que Tom peut gagner.',
 'Combien de temps sont-ils longs?',
 "Je n'arrive pas à croire que ta mère vous aie laissé partir.",
 'Penses-tu que je puisse utiliser mon téléphone dans la douche?',
 "Laissez-moi t'embrasser une dernière fois.",
 "J'espère que personne ne t'a suivi.",
 "Nous devons prendre ce problème à propos de l'extérieur.",
 "Vous n'

In [15]:
test_df['translated_texts'] = translated_texts
test_df

,src,tar,translated_texts
0,They say that Firefox is downloaded over 8 mil...,"Ils disent que chaque jour, Firefox est téléch...",On dit que les qualités de téléphone sont très...
1,I really like being with you.,J'apprécie vraiment d'être avec toi.,J'aime vraiment être avec vous.
2,"When I'm hot, a glass of cool water really ref...","Lorsque j'ai chaud, un verre d'eau fraîche me ...","Lorsque je suis chaud, un verre d'eau fraîche-..."
3,That's what I'm supposed to be doing.,C'est ce que je suis censée être en train de f...,C'est ce que je suis censé faire.
4,"She was superstitious, as the people of that p...","Elle était superstitieuse, à l'instar des gens...","C'était une grosse personne extraordinaire, pl..."
...,...,...,...
23269,You dropped something.,Vous avez laissé tomber quelque chose.,Tu as fait tomber quelque chose.
23270,I need you right now.,J'ai besoin de vous sur-le-champ.,J'ai besoin de toi maintenant.
23271,The weather service has issued a severe weathe...,Le service météorologique a émis un avis de te...,Le temps depuis afin évident un sévère sévèrem...
23272,Let's do the math.,Faisons le calcul.,Faisons les maths.


In [16]:
test_df.to_csv('final_prediction.csv', index=False)